In [184]:
import re
import sys
import requests


import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from tqdm import tqdm

from pymystem3 import Mystem

# from scipy.sparse import hstack

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split

### Импортируем датасет и предобученную модель русского языка

In [79]:
%%time
dataframe = pd.read_excel('/Users/magomednikolaev/Documents/Работа/NLP/we/comments.xlsx')
russian_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/magomednikolaev/Documents/Работа/NLP/we/we/186/model.txt', binary=False)

CPU times: user 52.2 s, sys: 1.66 s, total: 53.8 s
Wall time: 57.4 s


### Обучим модель на уже имеющихся данных 

Удалим не имеющие отношения к делу атрибуты

In [150]:
del dataframe['text_init']
del dataframe['text_init_TOK']
del dataframe['bast_list']
del dataframe['word_cnt']

dataframe = dataframe[['text_TOK_str', 'rating_cat', 'is_informative']]
dataframe = dataframe.rename(columns = {'text_TOK_str' : 'text',
                          'rating_cat' : 'tone',
                          'is_informative' : 'informative'})

Пока возьмем только тональность 

In [68]:
dataframeTone = dataframe.drop({'informative'}, axis = 1) 
dataframeInf = dataframe.drop({'tone'}, axis = 1)

In [76]:
dataframeTone_train, dataframeTone_test = train_test_split(dataframeTone, test_size = 0.2, random_state = 42)

In [77]:
len(X_train), len(X_test)

(4176, 1045)

Будем использовать tf-idf vectorizer, ранее эта модель показала наилучшие параметры при ngram_range=(1, 4), поэтому пока остаивим этот параметр 

In [81]:
text_transformer = TfidfVectorizer(
                                   ngram_range=(1, 4), 
                                   lowercase=True, max_features=150000)

In [82]:
%%time
X_train = text_transformer.fit_transform(dataframeTone_train['text'])
X_test = text_transformer.transform(dataframeTone_test['text'])
y_train = dataframeTone_train['tone']
y_test = dataframeTone_test['tone']

CPU times: user 1.04 s, sys: 69.3 ms, total: 1.11 s
Wall time: 1.21 s


In [84]:
X_train.shape, X_test.shape

((4176, 150000), (1045, 150000))

В качестве модели возьмем логистическую регрессию

In [90]:
LogReg = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
LogReg.fit(X_train, y_train)
y_pred = LogReg.predict(X_test)

In [92]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.85      0.92      0.88       496
     class 1       0.89      0.82      0.85       403
     class 2       0.88      0.79      0.83       146

    accuracy                           0.87      1045
   macro avg       0.87      0.85      0.86      1045
weighted avg       0.87      0.87      0.86      1045



/Users/magomednikolaev/opt/anaconda3/envs/nlp/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Теперь попробуем аугментировать данные и посмотрим на качество модели после этого

Напишем функцию(сопрем с гитхаба: https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_mystem.py), которая будет переводить mystem tags в upos tags, которые используются в нужных моделях. Несуществующие слова просто пропускаются, также, как и знаки препинания. 

In [146]:
def tag_mystem(
    text="Текст нужно передать функции в виде строки!", mapping=None, postags=True
):
    # если частеречные тэги не нужны (например, их нет в модели), выставьте postags=False
    # в этом случае на выход будут поданы только леммы

    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(",")[0]
            pos = pos.split("=")[0].strip()
            if mapping:
                if pos in mapping:
                    pos = mapping[pos]  # здесь мы конвертируем тэги
                else:
                    pos = "X"  # на случай, если попадется тэг, которого нет в маппинге
            tagged.append(lemma.lower() + "_" + pos)
        except KeyError:
            continue  # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
        except IndexError:
            continue # я здесь пропускаю слова с опечатками и просто несуществующие
            
    if not postags:
        tagged = [t.split("_")[0] for t in tagged]
    return tagged


# Таблица преобразования частеречных тэгов Mystem в тэги UPoS:
mapping_url = "https://raw.githubusercontent.com/akutuzov/universal-pos-tags/4653e8a9154e93fe2f417c7fdb7a357b7d6ce333/ru-rnc.map"

mystem2upos = {}
r = requests.get(mapping_url, stream=True)
for pair in r.text.split("\n"):
    pair = pair.split()
    if len(pair) > 1:
        mystem2upos[pair[0]] = pair[1]

print("Loading the model...", file=sys.stderr)
m = Mystem()

print("Processing input...", file=sys.stderr)
for line in sys.stdin:
    res = line.strip()
    output = tag_mystem(text=res, mapping=mystem2upos)
    print(" ".join(output))

Loading the model...
Processing input...


Посмотрим на возможные тэги

In [206]:
mystem2upos.values()

dict_values(['ADJ', 'ADV', 'ADV', 'ADJ', 'DET', 'ADJ', 'SCONJ', 'INTJ', 'X', 'NUM', 'PART', 'ADP', 'NOUN', 'PRON', 'X', 'VERB'])

Присвоим тэг всему тексту, каждому слову

In [157]:
dataframeAug = dataframe.copy()
dataframeAug.text = dataframeAug.text.apply(lambda x: tag_mystem(x, mapping = mystem2upos))

Посмотрим для начала, с какими проблемами может столкунться модель:
   1. Уже на этапе тэга не все слова могут найтись
   2. Даже по правильным тэгам не может найти 
   3. После применения замены появпялются "-"
   4. Слова с опечатками не проходят на одном из этапов: на моменте тэга(но может и пройти), на моменте поиска синонимичного слова(там уже при наличии ошибкии тэга, скорее всего ничего не сработает)

Саугментируем данный корпус и соеденим два датасета

In [189]:
# dataframeAug.loc[1000, 'text'][0]    
def replacement_by_WE(text):
    
    new_text = list()
    for word in text:
        try:
            new_word = russian_model.most_similar(word)[0][0].split('_')[0]
            new_word = re.sub('-', '', new_word)
        except KeyError:
            new_word = word.split("_")[0]
        new_text.append(new_word)
    return new_text  

# dataframeAug.text.apply(lambda x: replacement_by_WE(x))

In [216]:
new_corpus = list()
for text in tqdm(dataframeAug.text):
    
    new_text = list()
    for word in text:
        try:
            new_word = russian_model.most_similar(word)[0][0].split('_')[0]
            new_word = re.sub('-', '', new_word)
        except KeyError:
            new_word = word.split("_")[0]
            
        new_text.append(new_word)
    new_corpus.append(' '.join(new_text))

100%|███████████████████████████████████████| 5221/5221 [36:08<00:00,  2.41it/s]


Чепуха:
  1. ,д
  2. ,з
  3. .таки
  4. ...ть
  5. б;льший

In [231]:
dataframeAug['text'] = new_corpus

In [263]:
dataframe_concated = pd.concat([dataframeAug, dataframe])
dataframe_concated = dataframe_concated.drop_duplicates(subset=['text'])

### Переобучим модель на аугментированных данных 

In [264]:
dataframe_concatedTone = dataframe_concated.drop({'informative'}, axis = 1) 
dataframe_concatedInf = dataframe_concated.drop({'tone'}, axis = 1)

In [265]:
dataframe_concatedTone_train, dataframe_concatedTone_test = train_test_split(dataframe_concatedTone, 
                                                                             test_size = 0.2, 
                                                                             random_state = 42)

In [266]:
text_transformer = TfidfVectorizer(
                                   ngram_range=(1, 4), 
                                   lowercase=True, 
                                    max_features=150000)

In [267]:
%%time
Xaug_train = text_transformer.fit_transform(dataframe_concatedTone_train['text'])
Xaug_test = text_transformer.transform(dataframe_concatedTone_test['text'])
yaug_train = dataframe_concatedTone_train['tone']
yaug_test = dataframe_concatedTone_test['tone']

CPU times: user 2.23 s, sys: 218 ms, total: 2.45 s
Wall time: 2.54 s


In [268]:
LogReg = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
LogReg.fit(Xaug_train, yaug_train)
yuag_pred = LogReg.predict(Xaug_test)

In [269]:
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(yaug_test, yuag_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.87      0.94      0.90       956
     class 1       0.92      0.87      0.89       884
     class 2       0.94      0.85      0.89       230

    accuracy                           0.90      2070
   macro avg       0.91      0.88      0.90      2070
weighted avg       0.90      0.90      0.90      2070



Таким образом, модель стала работать лучше, хотя возможно, она переобучилась. Далее прдестоит провести кросс валидацию, попробовать изменять гипер-параметры и стратегию аугментации.
Полезные ссылки:
1. https://www.kaggle.com/theoviel/using-word-embeddings-for-data-augmentation
2. http://www.machinelearning.ru/wiki/images/b/b3/Word2Vec.pdf
3. https://github.com/akutuzov/webvectors
4. https://universaldependencies.org/u/pos/

Возможно, она будет работаь еще хуже, если агументировать отдельно и тестовую выборку и трейн выборки и уже сранвитьвать метрики по одним и тем же выборкам